# Using the simulator

In this notebook you will learn how to use the simulator to test out methods.

## Setting up a connection with the robot

As described in the [basic liquid handling tutorial](basic), we will use the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class to control the robot. This time, however, instead of using the Hamilton {class}`~pylabrobot.liquid_handling.backends.hamilton.STAR` backend, we are using virtual the {class}`~pylabrobot.liquid_handling.backends.simulation.simulator_backend.SimulatorBackend` backend. This means that the liquid handling protocol will look exactly the same, but the commands are sent to the simulator instead of a real physical robot.

In [1]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.simulation.simulator_backend import SimulatorBackend

Again, we use the {class}`~pylabrobot.resources.hamilton.STARLetDeck`:

In [2]:
from pylabrobot.resources.hamilton import STARLetDeck

Instantiate the backend:

In [3]:
sb = SimulatorBackend()

dir(sb)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_assemble_command',
 '_fst',
 '_generate_id',
 '_httpd',
 '_id',
 '_loop',
 '_num_channels',
 '_replay',
 '_run_file_server',
 '_sent_messages',
 '_socket_handler',
 '_stop_',
 '_t',
 '_websocket',
 'adjust_well_volume',
 'aspirate',
 'aspirate96',
 'assigned_resource_callback',
 'clear_tips',
 'deserialize',
 'dispense',
 'dispense96',
 'drop_tips',
 'drop_tips96',
 'edit_tips',
 'fill_tip_rack',
 'fs_host',
 'fs_port',
 'fst',
 'handle_event',
 'has_connection',
 'httpd',
 'loop',
 'move_channel_x',
 'move_channel_y',
 'move_channel_z',
 'move_resource',


Instantiate the "liquid handler" class, by passing it the instantiated "backend", and an evaluated "deck" object.

In [4]:
dk = STARLetDeck()

lh = LiquidHandler(backend=sb, deck=dk)

dir(lh)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_assert_positions_unique',
 '_assert_resources_exist',
 '_check_args',
 '_picked_up_tips96',
 'aspirate',
 'aspirate_plate',
 'backend',
 'clear_head_state',
 'deck',
 'deserialize',
 'discard_tips',
 'dispense',
 'dispense_plate',
 'drop_tips',
 'drop_tips96',
 'get_resource',
 'head',
 'load',
 'move_lid',
 'move_plate',
 'move_resource',
 'pick_up_tips',
 'pick_up_tips96',
 'resource_assigned_callback',
 'resource_unassigned_callback',
 'return_tips',
 'return_tips96',
 'save',
 'serialize',
 'setup',
 'setup_finished',
 'stamp',
 'stop',
 'summary',
 'transfer',
 'unassign_resou

Calling {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.setup` will set up the simulation server and open it in a new browser tab.

In [5]:
lh.setup()

Simulation server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337. Open this URL in your browser.


ValueError: Cannot wait for response when no websocket connection is established.

Call {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.wait_for_connection` to wait for the simulator to connect to the server. This may take a few seconds. (Be sure to open the file server link in the browser if it doesn't open automatically!)

In [6]:
sb.wait_for_connection()

print("Done")

Done


For the optimal experience, we recommend that you run the notebook and simulator side by side.

![The empty simulator](./img/simulator/empty.png)

## Assigning plates and tips

With the simulator, {func}`assigning resources <pylabrobot.resources.hamilton.STARLetDeck.assign_child_resource>` has the additional affect of placing the resources on the simulated deck. They will appear immediately.

In [7]:
from pylabrobot.resources import (
  TIP_CAR_480_A00,
  PLT_CAR_L5AC_A00,
  Cos_96_DW_1mL,
  STF_L
)

In [8]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
for i in range(2):
    tip_car[i] = STF_L(name=f'tip_rack_{i}')

In [9]:
lh.deck.assign_child_resource(tip_car, rails=1, replace=True)

When you set or delete subresources in a {class}`~pylabrobot.resources.abstract.Carrier`, that will immediately be reflected in the UI.

In [10]:
tip_car[2] = STF_L(name='tip_rack_2')

In [11]:
tip_car[3] = STF_L(name='tip_rack_3')

In [12]:
del tip_car[3]

In [13]:
tip_car[4] = STF_L(name='tip_rack_4')

In [14]:
tip_car[4] = None

In [15]:
tip_car[3] = None

In [16]:
tip_car[3] = STF_L(name='tip_rack_3')

In [17]:
tip_car[4] = STF_L(name='tip_rack_4')

In [18]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
for i in range(5):
    plt_car[i] = Cos_96_DW_1mL(name=f'plate_{i}')

In [19]:
lh.deck.assign_child_resource(plt_car, rails=9, replace=True)

![The simulator after the resources have been assigned](./img/simulator/assignment.png)

## Build the deck layout: placing resources

Where you would manually place the resources like tips and liquid on the deck when using a physical system, with the Simulator you can add them using code.

### Tips

Let's {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.edit_tips` the tip racks to place tips. You can also use this method to remove tips.

In [20]:
tip_rack_0 = lh.get_resource("tip_rack_0")
sb.fill_tip_rack(tip_rack_0)

In [21]:
tip_rack_1 = lh.get_resource("tip_rack_1")
sb.fill_tip_rack(tip_rack_1)

In [22]:
tip_rack_2 = lh.get_resource("tip_rack_2")
sb.edit_tips(tip_rack_2, pattern=[[True, True, False, False]*3]*8)

In [23]:
tip_rack_3 = lh.get_resource("tip_rack_3")
sb.edit_tips(tip_rack_3, pattern=[[True, False]*6]*8)

In [24]:
tip_rack_4 = lh.get_resource("tip_rack_4")
sb.edit_tips(tip_rack_4, pattern=[[True]*6 + [False]*6]*8)

### Liquids

Adding liquid to wells works similarly. You can use {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.adjust_well_volume` to adjust the volume of individual wells in each resource. Note that the opacity of the well matches the volume of the well.

In [25]:
plate_2 = lh.get_resource("plate_2")
sb.adjust_well_volume(plate_2, pattern=[[100, 800]*6]*8)

In [26]:
plate_1 = lh.get_resource("plate_1")
sb.adjust_well_volume(plate_1, pattern=[[1000]*12]*8)

In [27]:
plate_0 = lh.get_resource("plate_0")
sb.adjust_well_volume(plate_0, pattern=[[0]*12]*8)

![Simulator after the tips have been placed and the volumes adjusted](./img/simulator/resources.png)

## Liquid handling

Once the layout is complete, you can run the same commands as described in the [basic liquid handling tutorial](basic).

### Picking up tips

In [28]:
lh.pick_up_tips(tip_rack_0["A1:C1"])

In [29]:
lh.discard_tips(tip_rack_0["A1:C1"])

KeyError: TipSpot(name=tip_rack_0_tipspot_0_0, location=(007.200, 068.300, -50.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)

### Aspirating and dispensing

In [30]:
lh.pick_up_tips(tip_rack_0["A1"])

TipSpotHasNoTipError: Tip spot has no tip.

In [ ]:
lh.aspirate(plate_2["A2"], vols=[300])

In [ ]:
plate_4 = lh.get_resource("plate_4")
lh.dispense(plate_4["A2"], vols=[300])

In [ ]:
lh.discard_tips(tip_rack_0["A1"])

### Aspirating using CoRe 96

The CoRe 96 head supports liquid handling operations for 96 channels at once. Here's how to use:

- {func}`~pylabrobot.liquid_handling.LiquidHandler.pick_up_tips_96` for picking up 96 tips;
- {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate_plate` for aspirating liquid from the deck, where the `pattern` array controls which wells are aspirated from;
- {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense_plate` for dispensing liquid to the deck, where the `pattern` array controls which wells are dispensed to, and;
- {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips96` for discarding tips to the resource.


In [ ]:
lh.pick_up_tips96(tip_rack_0)

In [ ]:
lh.aspirate_plate(plate_1, volume=400)

In [ ]:
lh.dispense_plate(plate_0, volume=400)

In [ ]:
lh.aspirate_plate(plate_1, volume=400)

In [ ]:
lh.dispense_plate(plate_0, volume=400)

In [ ]:
lh.discard_tips96(tip_rack_0)

![The simulator after the liquid handling operations completed](./img/simulator/after_lh.png)

## Shutting down

When you're done, remember to shut down the simulator by calling {func}`~pylabrobot.liquid_handling.LiquidHandler.stop`.

In [ ]:
lh.stop()